In [ ]:
import pandas as pd
from pathlib import Path
import numpy as np
from sklearn.model_selection import train_test_split, KFold
import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV,GroupShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GroupKFold
# Import the necessary metric
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import shap
from scipy.stats import mannwhitneyu
import seaborn as sns

### Raw Read Count data (Getting it ready)

In [ ]:
dir_path = Path.cwd().parent

## Deal with Braken
path_braken = dir_path/'SHINE_Bracken_Data/All_SHINE_Braken_tidy_read_data.csv'
braken=pd.read_csv(path_braken)
braken_df = pd.DataFrame(braken)
contamination = braken_df.filter(regex='virus|Homo sapiens') #remove contamination
braken_df = braken_df.drop(columns=contamination.columns)



## Deal with the meta data
path2 = dir_path/'SHINE_Bracken_Data/MP_Data_SHINE_ID_Patient_ID_no_dups.csv'
Meta_data = pd.read_csv(path2)
Meta_data_df = pd.DataFrame(Meta_data)
Meta_data_df =Meta_data_df[(Meta_data_df.duplicated(subset=['SubjectID_child', 'visitlabel'], keep='first'))|(~Meta_data_df.duplicated(subset=['SubjectID_child', 'visitlabel'], keep=False))].reset_index(drop=True)
Meta_data_df

#merge Meta_data_df with Braken read data
Braken_Meta_merge_df = Meta_data_df.merge(braken_df, left_on='run_accession', right_on='sample')
Braken_Meta_merge_df = Braken_Meta_merge_df[(Braken_Meta_merge_df.duplicated(subset=['SubjectID_child', 'visitlabel'], keep='first'))|(~Braken_Meta_merge_df.duplicated(subset=['SubjectID_child', 'visitlabel'], keep=False))].drop(columns='sample').reset_index(drop=True)
Dominant_species_path = pd.read_csv(dir_path/'SHINE_Bracken_Data/Dominant_Species_All_Bracken_G8tr1.csv')
Dominant_species_df = pd.DataFrame(Dominant_species_path)

Braken_meta_dominant_df = Braken_Meta_merge_df[['run_accession','SubjectID_child','visitlabel']+list(Dominant_species_df.columns)] #dataframe of braken dom species with patient id and ERR#


#find Ecoli Coptr samples
Coptr_path = pd.read_csv(dir_path/'SHINE_COPTR_Data/Coptr_tiddy_df.csv')
All_Coptr_df = pd.DataFrame(Coptr_path)
Coptr_Meta_merge_df = Meta_data_df.drop(columns=['SubjectID_child','SHINE_ID','visitlabel','age','age_group']).merge(All_Coptr_df, left_on='run_accession',right_on='Sample')
Coptr_Meta_merge_df.drop(columns='run_accession',inplace=True)

#Get Ecoli samples 
Samples_w_Your_Species_rates = Coptr_Meta_merge_df[Coptr_Meta_merge_df['Escherichia coli']>0]['Sample'] # change to species name that you want 
Coptr_Ecoli_df = Coptr_Meta_merge_df[Coptr_Meta_merge_df['Sample'].isin(Samples_w_Your_Species_rates)]
#Coptr_Ecoli_GR = Coptr_Ecoli_df['Escherichia coli']
#Coptr_Ecoli_GR_df = pd.DataFrame(Coptr_Ecoli_GR)
#Coptr_Ecoli_GR_df.reset_index(drop=True, inplace = True)



#match with Braken
Samples_w_Your_Species_Braken_data_df = Braken_meta_dominant_df[Braken_meta_dominant_df['run_accession'].isin(Samples_w_Your_Species_rates)]
#put together 
All_Coptr_E_Coli_GR = Coptr_Ecoli_df[['Sample','Escherichia coli']]  #dataframe of sample number and Ecoli Growth rate 
our_coptr_EC_GR = All_Coptr_E_Coli_GR[All_Coptr_E_Coli_GR['Sample'].isin(Samples_w_Your_Species_Braken_data_df['run_accession'])] ##make sure we get Coptr data(ecoli gr samples) that are in our braken data 

Full_data = Samples_w_Your_Species_Braken_data_df.merge(our_coptr_EC_GR, left_on = 'run_accession', right_on='Sample').drop(columns='Sample').rename(columns={'Escherichia coli_y':'Ecoli_GR'})

Full_data_copy = Full_data.copy()
psuedocount = 0.1
Full_data_copy = Full_data_copy.applymap(lambda x: psuedocount if x ==0 else x)

Full_data_copy.drop(columns= ['Escherichia phage vB_EcoS-Ro145c2YLVW','Escherichia phage vB_EcoS_fFiEco03','Escherichia phage 26','Escherichia phage SZH-1','Salmonella phage vB_SAg-RPN213'],inplace=True)

Full_data_copy.columns = Full_data_copy.columns.str.replace(r'[^A-Za-z0-9_]', '', regex=True)
Full_data_copy.columns = Full_data_copy.columns.str.replace(' ', '_')
Full_data_copy = Full_data_copy.loc[:, ~Full_data_copy.columns.duplicated()]
Full_data_copy 
#psuedocount = 0.1
#Samples_w_Your_Species_Braken_w_psuedo = Samples_w_Your_Species_Braken_data_df_copy.applymap(lambda x: psuedocount if x ==0 else x)
#Samples_w_Your_Species_Braken_data_a

In [ ]:
Full_data.to_csv('Full_data.csv',index=False)

In [ ]:
Full_data_copy.to_csv('Full_data_copy.csv',index=False)

In [ ]:
list(Full_data_copy.columns)

In [ ]:
#np.linspace(0.01, 0.1, 10)
#np.arange(50, 500, 50)
np.arange(0,.5,.01)
np.linspace(0.01, 0.07, 14)
np.arange(0.01, 0.07, .001)

# Train/Test Split Strategies


### Random Split

In [ ]:
def Random_split(data,number):
    # Separate the data into features and target (X and Y)
    X = data.drop(columns=['run_accession', 'SubjectID_child', 'visitlabel', 'Ecoli_GR'])
    # Target variable (Ecoli_Growth_rate)
    Y = data['Ecoli_GR']
    
    # Set up lists for model performance
    best_models = []   
    validation_score_list = []
    test_scores_list = []
    r2_list = []
    
    # Set up lists for the random model
    random_best_models = []
    random_valid_score_list = []
    random_test_scores_list = []
    random_r2_list = []


    # Define hyperparameters grid for the real model
    param_grid = {
        'num_leaves': np.arange(2, 25, 2),  # Range from 2 to 24 with a step of 2
        'max_depth': np.arange(2, 30),  # Range from 1 to 10
        'min_split_gain': np.arange(0,.5,.01),
        'learning_rate': np.arange(0.01, 0.07, .001),  # 10 evenly spaced values between 0.01 and 0.1
        'n_estimators': np.arange(50, 500, 50),  # Range from 50 to 200 with a step of 50'num_leaves': [2,4,6,8,10,12,14,16,24],
        'min_child_samples': np.arange(5,20,1),
        'subsample': [0.8],
        'colsample_bytree': [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1],
        'subsample_freq': [2,3,5,10],
        'reg_alpha': np.arange(0,.5,0.01),
        'reg_lambda': np.arange(0,.5,0.01),
        }
    
    # Goal: Shuffle data and randomly split Tmp and test data 10 times
    for i in range(1, number+1):
        # Shuffle and randomly split into Tmp and Test data 
        X_Tmp, X_Test, Y_Tmp, Y_Test = train_test_split(X, Y, test_size=0.2, train_size=0.8, random_state=None, shuffle=True)
        
        # Permute the target variable for the random model
        Y_tmp_perm = np.random.permutation(Y_Tmp)
        
        # Make our real model
        lgb_model = lgb.LGBMRegressor(boosting_type='gbdt', objective='regression')

        # Set up GridSearchCV for the real model
        random_search = RandomizedSearchCV(
            estimator=lgb_model,
            param_distributions=param_grid,
            n_iter=300,
            scoring='neg_mean_squared_error',
            cv=5,
            verbose=0,
            n_jobs=-1)
        
        # Fit the real model
        random_search.fit(X_Tmp, Y_Tmp)
        best_models.append(random_search.best_estimator_)
        
        # Compute SHAP values for the real model
        explainer_real = shap.Explainer(best_models[-1])  # Create a SHAP explainer
        shap_values_real = explainer_real(X_Test)  # Calculate SHAP values
        
        # Save SHAP values for the real model
        np.save(f'RandomSplit_shap_values_real_round_{i}.npy', shap_values_real.values)  # Save SHAP values
        # Optionally, save the actual data for inspection
        np.save(f'shap_data_round_{i}.npy', X_Test)

        # Save summary plot for the real model
        plt.figure()
        shap.summary_plot(shap_values_real, X_Test, show=False)
        plt.title(f'RandomSplit SHAP Summary Plot - Real Model - Round {i}')
        plt.savefig(f'Random_split_shap_summary_real_round_{i}.png')
        plt.close()

        # Collect scores for the real model
        Y_pred_real = random_search.best_estimator_.predict(X_Test)
        testing_mse = mean_squared_error(Y_Test, Y_pred_real)
        test_scores_list.append(testing_mse)
        r2 = r2_score(Y_Test, Y_pred_real)
        r2_list.append(r2)
        validation_score = random_search.best_score_
        validation_score_list.append(validation_score)
        Residuals = Y_Test - Y_pred_real

        # Make our random model
        random_model = lgb.LGBMRegressor(boosting_type='gbdt', objective='regression')
        
        # Set up GridSearchCV for the random model
        random_search_rm = RandomizedSearchCV(
            estimator=random_model,
            param_distributions=param_grid,
            scoring='neg_mean_squared_error',
            cv=5,
            verbose=0,
            n_jobs=-1
        )
        
        # Fit the random model
        random_search_rm.fit(X_Tmp, Y_tmp_perm)
        random_best_models.append(random_search_rm.best_estimator_)
        
        # Collect scores for the random model
        random_Y_pred = random_search_rm.best_estimator_.predict(X_Test)
        random_testing_mse = mean_squared_error(Y_Test, random_Y_pred)
        random_test_scores_list.append(random_testing_mse)
        random_r2 = r2_score(Y_Test, random_Y_pred)
        random_r2_list.append(random_r2)
        random_validation_score = random_search_rm.best_score_
        random_valid_score_list.append(random_validation_score)
        

        # Time of subplots!!!!
        fig, axs = plt.subplots(1, 2, figsize=(12, 5))  # Adjust figsize as needed

        # Plot for the real model
        axs[0].scatter(Y_Test, Y_pred_real, color='blue', label='Lightgbm', alpha=0.6)
        axs[0].scatter(Y_Test, random_Y_pred, color='orange', label='Random Model', alpha=0.6)
        axs[0].plot([Y_Test.min(), Y_Test.max()], [Y_Test.min(), Y_Test.max()], 'r--', lw=2)
        axs[0].set_title(f'Real Model - Round {i}')
        axs[0].set_xlabel('Actual Values')
        axs[0].set_ylabel('Predicted Values')
        axs[0].legend()
        axs[0].axis('equal')  # Set equal scaling

        # Plot for the real model
        axs[1].scatter(Y_pred_real,Residuals, color='blue', label='Residuals', alpha=0.6)
        axs[1].axhline(y=0, color='red', linestyle='--', lw=2)  # Horizontal line at 0
        axs[1].set_title(f'Real Model - Round {i}')
        axs[1].set_xlabel('Predicted Values')
        axs[1].set_ylabel('Residuals (Y_test - Y_pred)')
        axs[1].legend()
        axs[1].axis('equal')  # Set equal scaling
        axs[1].grid()

        # Show the plot
        plt.tight_layout()  # Adjust layout to prevent overlap
        plt.savefig(f'RandomSplit_plot_round_{i}.png')  # Save each plot as an image
        plt.show()
        plt.close()

    return best_models, validation_score_list, test_scores_list, r2_list, random_best_models, random_valid_score_list, random_test_scores_list, random_r2_list
    

In [ ]:
best_models, validation_score_list, test_scores_list, r2_list, random_best_models, random_valid_score_list, random_test_scores_list, random_r2_list = Random_split(Full_data_copy,10)
number=10
columns= np.arange(1,number+1,1)
RS_table = pd.DataFrame([validation_score_list,test_scores_list, r2_list],index=['validation_score','test_score','R2'],columns=columns)
Random_RS_model_table = pd.DataFrame([random_valid_score_list,random_test_scores_list,random_r2_list],index=['validation_score','test_score','R2'],columns=columns)


In [ ]:
RS_table

In [ ]:
r2_list

In [ ]:
best_models[4].get_params()

In [ ]:
best_models[0].get_params()

In [ ]:
best_models[1].get_params()

In [ ]:

# Extracting validation scores and test scores for both models
validation_scores = RS_table.loc['validation_score'].values
validation_scores = [-score for score in validation_score_list]
random_validation_scores = Random_RS_model_table.loc['validation_score'].values
random_validation_scores = [-score for score in random_valid_score_list]
test_scores = RS_table.loc['test_score'].values
random_test_scores = Random_RS_model_table.loc['test_score'].values

# Perform Mann-Whitney U test on validation scores
u_statistic_validation, p_value_validation = mannwhitneyu(validation_scores, random_validation_scores)

# Perform Mann-Whitney U test on test scores
u_statistic_test, p_value_test = mannwhitneyu(test_scores, random_test_scores)

# Print results
print(f'Mann-Whitney U Test on Validation Scores: U-statistic = {u_statistic_validation}, p-value = {p_value_validation}')
print(f'Mann-Whitney U Test on Test Scores: U-statistic = {u_statistic_test}, p-value = {p_value_test}')


In [ ]:
# Prepare the data for visualization
data = {
    'Model Type': ['LightGBM'] * len(validation_scores) + ['Random Model'] * len(random_validation_scores),
    'Validation Score': np.concatenate([validation_scores, random_validation_scores]),
    'Test Score': np.concatenate([test_scores, random_test_scores])
}

# Create a DataFrame
scores_df = pd.DataFrame(data)

# Set the aesthetics for the plots
sns.set(style="whitegrid")

# Create a boxplot for Validation Scores
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
sns.boxplot(x='Model Type', y='Validation Score', data=scores_df)
plt.title('Validation Data Performance')
plt.ylabel('Mean Squared Error')
plt.xlabel('Model Type')

# Create a boxplot for Test Scores
plt.subplot(1, 2, 2)
sns.boxplot(x='Model Type', y='Test Score', data=scores_df)
plt.title('Performance on Testing Data')
plt.ylabel('Mean Squared Error')
plt.xlabel('Model Type')

plt.tight_layout()
plt.show()


In [ ]:

# Create a violin plot for Validation Scores
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
sns.violinplot(x='Model Type', y='Validation Score', data=scores_df)
plt.title('Validation Scores Violin Plot')
plt.ylabel('Validation Score')
plt.xlabel('Model Type')

# Create a violin plot for Test Scores
plt.subplot(1, 2, 2)
sns.violinplot(x='Model Type', y='Test Score', data=scores_df)
plt.title('Test Scores Violin Plot')
plt.ylabel('Test Score')
plt.xlabel('Model Type')

plt.tight_layout()
plt.show()

### Group by Patient ID

In [ ]:
def Group_by_Patient(data,number):
    # Separate the data into features and target (X and Y)
    X = data.drop(columns=['run_accession', 'SubjectID_child', 'visitlabel', 'Ecoli_GR'])
    # Target variable (Ecoli_Growth_rate)
    Y = data['Ecoli_GR']
    groups = data['SubjectID_child']
    
    # Set up lists for model performance
    best_models = []   
    validation_score_list = []
    test_scores_list = []
    r2_list = []
    
    # Set up lists for the random model
    random_best_models = []
    random_valid_score_list = []
    random_test_scores_list = []
    random_r2_list = []

    # Make our real model
    lgb_model = lgb.LGBMRegressor(boosting_type='gbdt', objective='regression')
    # Define hyperparameters grid for the real model
    param_grid = {
        'num_leaves': np.arange(2, 12, 2),  # Range from 2 to 24 with a step of 2
        'max_depth': np.arange(2, 30),  # Range from 1 to 10
        #'min_split_gain': np.arange(0,.5,.01),
        'learning_rate': np.arange(0.01, 0.07, .001),  # 10 evenly spaced values between 0.01 and 0.1
        'n_estimators': np.arange(50, 500, 50),  # Range from 50 to 200 with a step of 50'num_leaves': [2,4,6,8,10,12,14,16,24],
        'min_child_samples': np.arange(1,5,1),
        'subsample': [0.8],
        'colsample_bytree': [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1],
        'subsample_freq': [2,3,5,10],
        'reg_alpha': np.arange(0,.5,0.01),
        'reg_lambda': np.arange(0,.5,0.01),
        }
    
    # Goal: Shuffle data and randomly split Tmp and test data 10 times
    for i in range(1, number+1):
        # Shuffle and randomly split into Tmp and Test data 
        gss=GroupShuffleSplit(n_splits=1,test_size=0.2,train_size=0.7)
        
        train_idx, test_idx = next(gss.split(X, Y, groups))
        X_Tmp = X.loc[train_idx]
        X_Test = X.loc[test_idx]
        Y_Tmp = Y.loc[train_idx]
        Y_Test = Y.loc[test_idx]
        
        # Permute the target variable for the random model
        Y_tmp_perm = np.random.permutation(Y_Tmp)
        
        # Make our real model
        lgb_model = lgb.LGBMRegressor(boosting_type='gbdt', objective='regression')
        
        group_kfold = GroupKFold(n_splits=5)
        
        # Set up RandomizedSearchCV for the real model
        random_search = RandomizedSearchCV(
            estimator=lgb_model,
            param_distributions=param_grid,
            n_iter=300,
            scoring='neg_mean_squared_error',
            cv=group_kfold,  # Use the group_kfold instance here
            verbose=0,
            n_jobs=-1)

        # Fit the model (assuming X_Tmp, Y_Tmp, and groups are defined)
        random_search.fit(X_Tmp, Y_Tmp, groups=groups.loc[train_idx])
        best_models.append(random_search.best_estimator_)

        # Compute SHAP values for the real model
        explainer_real = shap.Explainer(best_models[-1])  # Create a SHAP explainer
        shap_values_real = explainer_real(X_Test)  # Calculate SHAP values
        
        # Save SHAP values for the real model
        np.save(f'GroupbyPatient_shap_values_real_round_{i}.npy', shap_values_real.values)  # Save SHAP values
        # Optionally, save the actual data for inspection
        np.save(f'GroupbyPatient_shap_data_round_{i}.npy', X_Test)
        ##np.save(f'GroupbyPatient_shap_data_round_{i}.npy', X_Test((indexes))
        
        # Save summary plot for the real model
        plt.figure()
        shap.summary_plot(shap_values_real, X_Test, show=False)
        plt.title(f'GroupbyPatient SHAP Summary Plot - Real Model - Round {i}')
        plt.savefig(f'GroupbyPatient_shap_summary_real_round_{i}.png')
        plt.close()
        
        # Collect scores for the real model
        Y_pred_real = random_search.best_estimator_.predict(X_Test)
        testing_mse = mean_squared_error(Y_Test, Y_pred_real)
        test_scores_list.append(testing_mse)
        r2 = r2_score(Y_Test, Y_pred_real)
        r2_list.append(r2)
        validation_score = random_search.best_score_
        validation_score_list.append(validation_score)
        Residuals = Y_Test - Y_pred_real
        

        # Make our random model
        random_model = lgb.LGBMRegressor(boosting_type='gbdt', objective='regression')
        
        # Set up RandomizedSearchCV for the real model
        random_search_rm = RandomizedSearchCV(
            estimator=lgb_model,
            param_distributions=param_grid,
            n_iter=300,
            scoring='neg_mean_squared_error',
            cv=group_kfold,  # Use the group_kfold instance here
            verbose=0,
            n_jobs=-1)

        
        # Fit the random model
        random_search_rm.fit(X_Tmp, Y_tmp_perm,groups=groups.loc[train_idx])
        random_best_models.append(random_search_rm.best_estimator_)
        
        # Collect scores for the random model
        random_Y_pred = random_search_rm.best_estimator_.predict(X_Test)
        random_testing_mse = mean_squared_error(Y_Test, random_Y_pred)
        random_test_scores_list.append(random_testing_mse)
        random_r2 = r2_score(Y_Test, random_Y_pred)
        random_r2_list.append(random_r2)
        random_validation_score = random_search_rm.best_score_
        random_valid_score_list.append(random_validation_score)
        

        # Time of subplots!!!!
        fig, axs = plt.subplots(1, 2, figsize=(12, 5))  # Adjust figsize as needed

        # Plot for the real model
        axs[0].scatter(Y_Test, Y_pred_real, color='blue', label='LightGBM', alpha=0.6)
        axs[0].scatter(Y_Test, random_Y_pred, color='orange', label='Random Model', alpha=0.6)
        axs[0].plot([Y_Test.min(), Y_Test.max()], [Y_Test.min(), Y_Test.max()], 'r--', lw=2)
        axs[0].set_title(f'Real Model - Round {i}')
        axs[0].set_xlabel('Actual Values')
        axs[0].set_ylabel('Predicted Values')
        axs[0].legend()
        axs[0].axis('equal')  # Set equal scaling

        # Plot for the real model
        axs[1].scatter(Y_pred_real,Residuals, color='blue', label='Residuals', alpha=0.6)
        axs[1].axhline(y=0, color='red', linestyle='--', lw=2)  # Horizontal line at 0
        axs[1].set_title(f'Real Model - Round {i}')
        axs[1].set_xlabel('Predicted Values')
        axs[1].set_ylabel('Residuals (Y_test - Y_pred)')
        axs[1].legend()
        axs[1].axis('equal')  # Set equal scaling
        axs[1].grid()

        # Show the plot
        plt.tight_layout()  # Adjust layout to prevent overlap
        plt.savefig(f'GroupbyPatient_plot_round_{i}.png')  # Save each plot as an image
        plt.show()
        plt.close()

    return best_models, validation_score_list, test_scores_list, r2_list, random_best_models, random_valid_score_list, random_test_scores_list, random_r2_list
    

In [ ]:
best_models_GP, validation_score_list_GP, test_scores_list_GP, r2_list_GP, random_best_models_GP, random_valid_score_list_GP, random_test_scores_list_GP, random_r2_list_GP = Group_by_Patient(Full_data_copy,10)
number=10
columns= np.arange(1,number+1,1)
GP_table = pd.DataFrame([validation_score_list_GP,test_scores_list_GP, r2_list_GP],index=['validation_score','test_score','R2'],columns=columns)
Random_GP_model_table = pd.DataFrame([random_valid_score_list_GP,random_test_scores_list_GP,random_r2_list_GP],index=['validation_score','test_score','R2'],columns=columns)


In [ ]:
GP_table

In [ ]:
best_models_GP[3]

In [ ]:
best_models_GP[1].get_params()

In [ ]:
# Extracting validation scores and test scores for both models
validation_scores = GP_table.loc['validation_score'].values
validation_scores = [-score for score in validation_score_list_GP]
random_validation_scores = Random_GP_model_table.loc['validation_score'].values
random_validation_scores = [-score for score in random_valid_score_list_GP]
test_scores = GP_table.loc['test_score'].values
random_test_scores = Random_GP_model_table.loc['test_score'].values

# Perform Mann-Whitney U test on validation scores
u_statistic_validation, p_value_validation = mannwhitneyu(validation_scores, random_validation_scores)

# Perform Mann-Whitney U test on test scores
u_statistic_test, p_value_test = mannwhitneyu(test_scores, random_test_scores)

# Print results
print(f'Mann-Whitney U Test on Validation Scores: U-statistic = {u_statistic_validation}, p-value = {p_value_validation}')
print(f'Mann-Whitney U Test on Test Scores: U-statistic = {u_statistic_test}, p-value = {p_value_test}')


In [ ]:
# Prepare the data for visualization
data = {
    'Model Type': ['LightGBM'] * len(validation_scores) + ['Random Model'] * len(random_validation_scores),
    'Validation Score': np.concatenate([validation_scores, random_validation_scores]),
    'Test Score': np.concatenate([test_scores, random_test_scores])
}

# Create a DataFrame
scores_df = pd.DataFrame(data)

# Set the aesthetics for the plots
sns.set(style="whitegrid")

# Create a boxplot for Validation Scores
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
sns.boxplot(x='Model Type', y='Validation Score', data=scores_df)
plt.title('Validation Data Performance')
plt.ylabel('Mean Squared Error')
plt.xlabel('Model Type')

# Create a boxplot for Test Scores
plt.subplot(1, 2, 2)
sns.boxplot(x='Model Type', y='Test Score', data=scores_df)
plt.title('Performance on Testing Data')
plt.ylabel('Mean Squared Error')
plt.xlabel('Model Type')

plt.tight_layout()
plt.show()

### Group by Time point 

In [ ]:
def Group_by_Time(data,number):
    # Separate the data into features and target (X and Y)
    X = data.drop(columns=['run_accession', 'SubjectID_child', 'visitlabel', 'Ecoli_GR'])
    # Target variable (Ecoli_Growth_rate)
    Y = data['Ecoli_GR']
    groups = data['visitlabel']
    
    # Set up lists for model performance
    best_models = []   
    validation_score_list = []
    test_scores_list = []
    r2_list = []
    
    # Set up lists for the random model
    random_best_models = []
    random_valid_score_list = []
    random_test_scores_list = []
    random_r2_list = []

    # Define hyperparameters grid for the real model
    param_grid = {
        'num_leaves': np.arange(2, 12, 2),  # Range from 2 to 24 with a step of 2
        'max_depth': np.arange(2, 30),  # Range from 1 to 10
        #'min_split_gain': np.arange(0,.5,.01),
        'learning_rate': np.arange(0.01, 0.07, .001),  # 10 evenly spaced values between 0.01 and 0.1
        'n_estimators': np.arange(50, 500, 50),  # Range from 50 to 200 with a step of 50'num_leaves': [2,4,6,8,10,12,14,16,24],
        'min_child_samples': np.arange(1,5,1),
        'subsample': [0.8],
        'colsample_bytree': [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1],
        'subsample_freq': [2,3,5,10],
        'reg_alpha': np.arange(0,.5,0.01),
        'reg_lambda': np.arange(0,.5,0.01),
        }
    
    # Goal: Shuffle data and randomly split Tmp and test data 10 times
    for i in range(1, number+1):
        # Shuffle and randomly split into Tmp and Test data 
        gss=GroupShuffleSplit(n_splits=1,test_size=0.2,train_size=0.8)
        
        train_idx, test_idx = next(gss.split(X, Y, groups))
        X_Tmp = X.loc[train_idx]
        X_Test = X.loc[test_idx]
        Y_Tmp = Y.loc[train_idx]
        Y_Test = Y.loc[test_idx]
        
        # Permute the target variable for the random model
        Y_tmp_perm = np.random.permutation(Y_Tmp)
        
        # Make our real model
        lgb_model = lgb.LGBMRegressor(boosting_type='gbdt', objective='regression')
        
        group_kfold = GroupKFold(n_splits=4)

        # Set up RandomizedSearchCV for the real model
        random_search = RandomizedSearchCV(
            estimator=lgb_model,
            param_distributions=param_grid,
            n_iter=300,
            scoring='neg_mean_squared_error',
            cv=group_kfold,  # Use the group_kfold instance here
            verbose=0,
            n_jobs=-1)

        
        # Fit the real model
        random_search.fit(X_Tmp, Y_Tmp,groups=groups.loc[train_idx])
        best_models.append(random_search.best_estimator_)

        # Compute SHAP values for the real model
        explainer_real = shap.Explainer(best_models[-1])  # Create a SHAP explainer
        shap_values_real = explainer_real(X_Test)  # Calculate SHAP values
        
        # Save SHAP values for the real model
        np.save(f'GroupbyTime_shap_values_real_round_{i}.npy', shap_values_real.values)  # Save SHAP values
        # Optionally, save the actual data for inspection
        np.save(f'GroupbyTime_shap_data_round_{i}.npy', X_Test)

        # Save summary plot for the real model
        plt.figure()
        shap.summary_plot(shap_values_real, X_Test, show=False)
        plt.title(f'GroupbyTime SHAP Summary Plot - Real Model - Round {i}')
        plt.savefig(f'GroupbyTime_shap_summary_real_round_{i}.png')
        plt.close()
        
        # Collect scores for the real model
        Y_pred_real = random_search.best_estimator_.predict(X_Test)
        testing_mse = mean_squared_error(Y_Test, Y_pred_real)
        test_scores_list.append(testing_mse)
        r2 = r2_score(Y_Test, Y_pred_real)
        r2_list.append(r2)
        validation_score = random_search.best_score_
        validation_score_list.append(validation_score)
        Residuals = Y_Test - Y_pred_real
        

        # Make our random model
        random_model = lgb.LGBMRegressor(boosting_type='gbdt', objective='regression')
        
        # Set up GridSearchCV for the random model
        random_search_rm = RandomizedSearchCV(
            estimator=lgb_model,
            param_distributions=param_grid,
            n_iter=300,
            scoring='neg_mean_squared_error',
            cv=group_kfold,  # Use the group_kfold instance here
            verbose=0,
            n_jobs=-1)
        
        # Fit the random model
        random_search_rm.fit(X_Tmp, Y_tmp_perm,groups=groups.loc[train_idx])
        random_best_models.append(random_search_rm.best_estimator_)
        
        # Collect scores for the random model
        random_Y_pred = random_search_rm.best_estimator_.predict(X_Test)
        random_testing_mse = mean_squared_error(Y_Test, random_Y_pred)
        random_test_scores_list.append(random_testing_mse)
        random_r2 = r2_score(Y_Test, random_Y_pred)
        random_r2_list.append(random_r2)
        random_validation_score = random_search_rm.best_score_
        random_valid_score_list.append(random_validation_score)
        

        # Time of subplots!!!!
        fig, axs = plt.subplots(1, 2, figsize=(12, 5))  # Adjust figsize as needed

        # Plot for the real model
        axs[0].scatter(Y_Test, Y_pred_real, color='blue', label='LightGBM', alpha=0.6)
        axs[0].scatter(Y_Test, random_Y_pred, color='orange', label='Random Model', alpha=0.6)
        axs[0].plot([Y_Test.min(), Y_Test.max()], [Y_Test.min(), Y_Test.max()], 'r--', lw=2)
        axs[0].set_title(f'Real Model - Round {i}')
        axs[0].set_xlabel('Actual Values')
        axs[0].set_ylabel('Predicted Values')
        axs[0].legend()
        axs[0].axis('equal')  # Set equal scaling

        # Plot for the real model
        axs[1].scatter(Y_pred_real,Residuals, color='blue', label='Residuals', alpha=0.6)
        axs[1].axhline(y=0, color='red', linestyle='--', lw=2)  # Horizontal line at 0
        axs[1].set_title(f'Real Model - Round {i}')
        axs[1].set_xlabel('Predicted Values')
        axs[1].set_ylabel('Residuals (Y_test - Y_pred)')
        axs[1].legend()
        axs[1].axis('equal')  # Set equal scaling
        axs[1].grid()
        
        # Show the plot
        plt.tight_layout()  # Adjust layout to prevent overlap
        plt.savefig(f'GroupbyTime_plot_round_{i}.png')
        plt.show()
        plt.close()

    return best_models, validation_score_list, test_scores_list, r2_list, random_best_models, random_valid_score_list, random_test_scores_list, random_r2_list
    

In [ ]:
best_models_GT, validation_score_list_GT, test_scores_list_GT, r2_list_GT, random_best_models_GT, random_valid_score_list_GT, random_test_scores_list_GT, random_r2_list_GT = Group_by_Time(Full_data_copy,10)
number=10
columns= np.arange(1,number+1,1)
GT_table = pd.DataFrame([validation_score_list_GT,test_scores_list_GT, r2_list_GT],index=['validation_score','test_score','R2'],columns=columns)
Random_GT_model_table = pd.DataFrame([random_valid_score_list_GT,random_test_scores_list_GT,random_r2_list_GT],index=['validation_score','test_score','R2'],columns=columns)


In [ ]:
GT_table

In [ ]:
random_best_models_GT[4].get_params()

In [ ]:
# Extracting validation scores and test scores for both models
validation_scores = GT_table.loc['validation_score'].values
validation_scores = [-score for score in validation_score_list_GT]
random_validation_scores = Random_GT_model_table.loc['validation_score'].values
random_validation_scores = [-score for score in random_valid_score_list_GT]
test_scores = GT_table.loc['test_score'].values
random_test_scores = Random_GT_model_table.loc['test_score'].values

# Perform Mann-Whitney U test on validation scores
u_statistic_validation, p_value_validation = mannwhitneyu(validation_scores, random_validation_scores)

# Perform Mann-Whitney U test on test scores
u_statistic_test, p_value_test = mannwhitneyu(test_scores, random_test_scores)

# Print results
print(f'Mann-Whitney U Test on Validation Scores: U-statistic = {u_statistic_validation}, p-value = {p_value_validation}')
print(f'Mann-Whitney U Test on Test Scores: U-statistic = {u_statistic_test}, p-value = {p_value_test}')


In [ ]:
# Prepare the data for visualization
data = {
    'Model Type': ['LightGBM'] * len(validation_scores) + ['Random Model'] * len(random_validation_scores),
    'Validation Score': np.concatenate([validation_scores, random_validation_scores]),
    'Test Score': np.concatenate([test_scores, random_test_scores])
}

# Create a DataFrame
scores_df = pd.DataFrame(data)

# Set the aesthetics for the plots
sns.set(style="whitegrid")

# Create a boxplot for Validation Scores
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
sns.boxplot(x='Model Type', y='Validation Score', data=scores_df)
plt.title('Validation Data Performance')
plt.ylabel('Mean Squared Error')
plt.xlabel('Model Type')

# Create a boxplot for Test Scores
plt.subplot(1, 2, 2)
sns.boxplot(x='Model Type', y='Test Score', data=scores_df)
plt.title('Performance on Testing Data')
plt.ylabel('Mean Squared Error')
plt.xlabel('Model Type')

plt.tight_layout()
plt.show()

### Group by Time point and patient

In [ ]:
Goal: 

In [ ]:

met_data = Full_data_copy[['run_accession','SubjectID_child','visitlabel']]

met_df = met_data.pivot(index='SubjectID_child', columns='visitlabel',values='run_accession')

#put in function form

#this will beed to be in a for loop to get multiple tmp,test 
met_df = met_df.sample(frac=1,random_state=42)
split_index = int(0.8*len(met_df))
train_df = met_df.iloc[:split_index]
test_df = met_df.iloc[split_index:]
train_M1_6_df = train_df.drop(columns=['M12','M18'])
Train_M1_6_df = train_M1_6_df.reset_index() 
err_numbers_train = Train_M1_6_df[['M01', 'M03', 'M06']].stack().tolist()

test_M1_6_df = test_df.drop(columns=['M01','M03','M06'])
Test_M1_6_df = test_M1_6_df.reset_index() 
err_numbers_test =Test_M1_6_df[['M12','M18']].stack().tolist()

train_df = Full_data_copy[Full_data_copy['run_accession'].isin(err_numbers_train)]
testing_df = Full_data_copy[Full_data_copy['run_accession'].isin(err_numbers_test)]

Train_df = train_df.drop(columns=['run_accession','SubjectID_child','visitlabel'])
Y_train_df = Train_df['Ecoli_GR']

Test_df = testing_df.drop(columns=['run_accession','SubjectID_child','visitlabel'])
Y_Test_df = Test_df['Ecoli_GR']




# Machine Learning Model with CV


In [ ]:
def Random_split(data,number):
    # Separate the data into features and target (X and Y)
    X = data.drop(columns=['run_accession', 'SubjectID_child', 'visitlabel', 'Ecoli_GR'])
    # Target variable (Ecoli_Growth_rate)
    Y = data['Ecoli_GR']
    
    # Set up lists for model performance
    best_models = []   
    validation_score_list = []
    test_scores_list = []
    r2_list = []
    
    # Set up lists for the random model
    random_best_models = []
    random_valid_score_list = []
    random_test_scores_list = []
    random_r2_list = []

     # Make our real model
    lgb_model = lgb.LGBMRegressor(boosting_type='gbdt', objective='regression')
    # Define hyperparameters grid for the real model
    param_grid = {
        'num_leaves': [2,4,6,8,10,12,14,16,24],
            #'max_depth': [10, 50, 100],
            #'learning_rate': [0.01, 0.03,.05],
            #'n_estimators': [50, 100,150],
    }
        
    # Set up GridSearchCV for the real model
    grid_search = GridSearchCV(
        estimator=lgb_model,
        param_grid=param_grid,
        scoring='neg_mean_squared_error',
        cv=5,
        verbose=0,
        n_jobs=-1
    )
    
    # Goal: Shuffle data and randomly split Tmp and test data 10 times
    for i in range(1, number+1):
        # Shuffle and randomly split into Tmp and Test data 
        X_Tmp, X_Test, Y_Tmp, Y_Test = train_test_split(X, Y, test_size=0.2, train_size=0.8, random_state=None, shuffle=True)
        
        # Permute the target variable for the random model
        Y_tmp_perm = np.random.permutation(Y_Tmp)
        
        # Fit the real model
        grid_search.fit(X_Tmp, Y_Tmp)
        best_models.append(grid_search.best_estimator_)
        
        # Compute SHAP values for the real model
        explainer_real = shap.Explainer(best_models[-1])  # Create a SHAP explainer
        shap_values_real = explainer_real(X_Test)  # Calculate SHAP values
        
        # Save SHAP values for the real model
        np.save(f'RandomSplit_shap_values_real_round_{i}.npy', shap_values_real.values)  # Save SHAP values
        # Optionally, save the actual data for inspection
        np.save(f'shap_data_round_{i}.npy', X_Test)

        # Save summary plot for the real model
        plt.figure()
        shap.summary_plot(shap_values_real, X_Test, show=False)
        plt.title(f'RandomSplit SHAP Summary Plot - Real Model - Round {i}')
        plt.savefig(f'Random_split_shap_summary_real_round_{i}.png')
        plt.close()

        # Collect scores for the real model
        Y_pred_real = grid_search.best_estimator_.predict(X_Test)
        testing_mse = mean_squared_error(Y_Test, Y_pred_real)
        test_scores_list.append(testing_mse)
        r2 = r2_score(Y_Test, Y_pred_real)
        r2_list.append(r2)
        validation_score = grid_search.best_score_
        validation_score_list.append(validation_score)
        Residuals = Y_Test - Y_pred_real

        # Make our random model
        random_model = lgb.LGBMRegressor(boosting_type='gbdt', objective='regression')
        
        # Set up GridSearchCV for the random model
        grid_search_rm = GridSearchCV(
            estimator=random_model,
            param_grid=param_grid,
            scoring='neg_mean_squared_error',
            cv=5,
            verbose=0,
            n_jobs=-1
        )
        
        # Fit the random model
        grid_search_rm.fit(X_Tmp, Y_tmp_perm)
        random_best_models.append(grid_search_rm.best_estimator_)
        
        # Collect scores for the random model
        random_Y_pred = grid_search_rm.best_estimator_.predict(X_Test)
        random_testing_mse = mean_squared_error(Y_Test, random_Y_pred)
        random_test_scores_list.append(random_testing_mse)
        random_r2 = r2_score(Y_Test, random_Y_pred)
        random_r2_list.append(random_r2)
        random_validation_score = grid_search_rm.best_score_
        random_valid_score_list.append(random_validation_score)
        

        # Time of subplots!!!!
        fig, axs = plt.subplots(1, 2, figsize=(12, 5))  # Adjust figsize as needed

        # Plot for the real model
        axs[0].scatter(Y_Test, Y_pred_real, color='blue', label='Lightgbm', alpha=0.6)
        axs[0].scatter(Y_Test, random_Y_pred, color='orange', label='Random Model', alpha=0.6)
        axs[0].plot([Y_Test.min(), Y_Test.max()], [Y_Test.min(), Y_Test.max()], 'r--', lw=2)
        axs[0].set_title(f'Real Model - Round {i}')
        axs[0].set_xlabel('Actual Values')
        axs[0].set_ylabel('Predicted Values')
        axs[0].legend()
        axs[0].axis('equal')  # Set equal scaling

        # Plot for the real model
        axs[1].scatter(Y_pred_real,Residuals, color='blue', label='Residuals', alpha=0.6)
        axs[1].axhline(y=0, color='red', linestyle='--', lw=2)  # Horizontal line at 0
        axs[1].set_title(f'Real Model - Round {i}')
        axs[1].set_xlabel('Predicted Values')
        axs[1].set_ylabel('Residuals (Y_test - Y_pred)')
        axs[1].legend()
        axs[1].axis('equal')  # Set equal scaling
        axs[1].grid()

        # Plot for the random model
        #axs[2].scatter(Y_Test, random_Y_pred, color='orange', label='Predicted', alpha=0.6)
        #axs[2].plot([Y_Test.min(), Y_Test.max()], [Y_Test.min(), Y_Test.max()], 'r--', lw=2)
        #axs[2].set_title(f'Random Model - Round {i}')
        #axs[2].set_xlabel('Actual Values')
        #axs[2].set_ylabel('Predicted Values')
        #axs[2].legend()
        #axs[2].axis('equal')  # Set equal scaling

        # Show the plot
        plt.tight_layout()  # Adjust layout to prevent overlap
        plt.savefig(f'RandomSplit_plot_round_{i}.png')  # Save each plot as an image
        plt.show()
        plt.close()

    return best_models, validation_score_list, test_scores_list, r2_list, random_best_models, random_valid_score_list, random_test_scores_list, random_r2_list
    

FOr psuedo count
additive smoothing (alpha can be anything )
laplace smoothing 

In [ ]:
def light_gbm_set_up():
    #Step 4: Set up our Modelllll!! Lightgbm
    lgb_model = lgb.LGBMRegressor(boosting_type='gbdt',objective='regression')
    # We have t odefine our hyperparameters grid (this is for our random search function)
    param_grid = {
        'num_leaves': [2, 4, 8],          # Number of leaves in each tree
        'max_depth': [10,50,100],           # Maximum tree depth (-1 for no limit)
        'learning_rate': [0.05, 0.1,0.2],# Learning rate to adjust how much to correct per iteration
        'n_estimators': [50, 100, 150],         # Number of trees (boosting iterations)
        #'min_child_samples': [10, 20, 30],       # Minimum number of samples in one leaf
        #'subsample': [0.7, 0.8, 1.0],            # Fraction of data to be used for training each tree
        #colsample_bytree': [0.7, 0.8, 1.0],     # Fraction of features to use for training each tre
    }
    return lgb_model,param_grid
    

In [ ]:
def model_fit(choice,data):
   
    if choice==1:
        X_Train, Y_Train, X_Test, Y_test, kf = Random_split(data)
        lgb_model,param_grid = light_gbm_set_up()
        # Set up GridSearchCV with LightGBM
        grid_search = GridSearchCV(
            estimator=lgb_model,        # LightGBM model
            param_grid=param_grid,   # Hyperparameter grid
            scoring='neg_mean_squared_error', # Metric to evaluate: negative mean squared error (since it's regression)
            cv=kf,                      # GroupKFold CV object (10 folds)
            verbose=0,                  # Print results as we go
            n_jobs=-1                   # Use all cores
        )
        # Fit the model
        grid_search.fit(X_Train_RS, Y_Train_RS)
        best_model = grid_search.best_estimator_
        Y_pred = best_model.predict(X.iloc[test_idx])
        
    if choice==2:
        train_idx,X_Train, Y_Train, X_Test, Y_test, kf,groups = Group_by_Patient(data)
        lgb_model,param_grid = light_gbm_set_up()
        grid_search = GridSearchCV(
            estimator=lgb_model,        # LightGBM model
            param_grid=param_grid,   # Hyperparameter grid
            scoring='neg_mean_squared_error', # Metric to evaluate: negative mean squared error (since it's regression)
            cv=kf,                      # GroupKFold CV object (10 folds)
            verbose=0,                  # Print results as we go
            n_jobs=-1                   # Use all cores
        )
        # Fit the model using Grid_searchSearchCV
        # Fit the model with early stopping
        # Note: RandomizedSearchCV will call `fit` for each parameter combination
        grid_search.fit(X_Train, Y_Train, 
                        groups=groups.iloc[train_idx])
        best_model = grid_search.best_estimator_
        Y_pred = best_model.predict(X.iloc[test_idx])
        
    if choice==3:
        train_idx,X_Train, Y_Train, X_Test, Y_test, kf,groups = Group_by_Timepoint(data)
        lgb_model,param_grid = light_gbm_set_up()
        grid_search = GridSearchCV(
            estimator=lgb_model,        # LightGBM model
            param_grid=param_grid,   # Hyperparameter grid
            scoring='neg_mean_squared_error', # Metric to evaluate: negative mean squared error (since it's regression)
            cv=kf,                      # GroupKFold CV object (10 folds)
            verbose=0,                  # Print results as we go
            n_jobs=-1                   # Use all cores
        )
        # Fit the model using Grid_searchSearchCV
        # Fit the model with early stopping
        # Note: RandomizedSearchCV will call `fit` for each parameter combination
        grid_search.fit(X_Train, Y_Train, 
                        groups=groups.iloc[train_idx])

        best_model = grid_search.best_estimator_
        Y_pred = best_model.predict(X.iloc[test_idx])

    return best_model, Y_pred
    


In [ ]:
def model_fit(choice, data):
    # Define the training and test data using different methods based on our choice
    if choice == 1:
        X_Train, Y_Train, X_Test, Y_test, kf = Random_split(data)
    elif choice == 2:
        train_idx, X_Train, Y_Train, X_Test, Y_test, kf, groups = Group_by_Patient(data)
    elif choice == 3:
        train_idx, X_Train, Y_Train, X_Test, Y_test, kf, groups = Group_by_Timepoint(data)

    # Model setup
    lgb_model, param_grid = light_gbm_set_up()

    # Set up GridSearchCV
    grid_search = GridSearchCV(
        estimator=lgb_model,
        param_grid=param_grid,
        scoring='neg_mean_squared_error',
        cv=kf,
        verbose=0,
        n_jobs=-1
    )

    # Fit the model using GridSearchCV
    grid_search.fit(X_Train, Y_Train, groups=groups.iloc[train_idx] if choice != 1 else None)

    # Training MSE
    best_model = grid_search.best_estimator_
    Y_train_pred = best_model.predict(X_Train)
    training_mse = mean_squared_error(Y_Train, Y_train_pred)

    # Cross-validation MSE
    cv_mse = -grid_search.best_score_

    # Testing MSE
    Y_pred = best_model.predict(X_Test)
    testing_mse = mean_squared_error(Y_test, Y_pred)
    r2 = r2_score(Y_test, Y_pred)
    
    return best_model, Y_pred, r2, training_mse, cv_mse, testing_mse


In [ ]:
best_model, Y_pred, r2, training_mse, cv_mse, testing_mse = model_fit(1,Full_data_copy)

# Print the results
print(f"Test Mean Squared Error (MSE): {testing_mse:.4f}")
print(f"Test R-squared (R²): {r2:.4f}")
print('Training mse:',training_mse)
print('CV mse:',cv_mse)

In [ ]:
best_model, Y_pred, r2, training_mse, cv_mse, testing_mse = model_fit(2,Full_data_copy)

# Print the results
print(f"Test Mean Squared Error (MSE): {testing_mse:.4f}")
print(f"Test R-squared (R²): {r2:.4f}")
print('Training mse:',training_mse)
print('CV mse:',cv_mse)

In [ ]:
best_model

In [ ]:
best_model, Y_pred, r2, training_mse, cv_mse, testing_mse = model_fit(3,Full_data_copy)

# Print the results
print(f"Test Mean Squared Error (MSE): {testing_mse:.4f}")
print(f"Test R-squared (R²): {r2:.4f}")
print('Training mse:',training_mse)
print('CV mse:',cv_mse)

In [ ]:
best_model

In [ ]:
import matplotlib.pyplot as plt
import lightgbm as lgb

# Get feature importances from the model
importance = best_model.feature_importances_

X = Full_data_copy.drop(columns=['run_accession', 'SubjectID_child', 'visitlabel', 'Ecoli_GR'])

feature_names = X.columns  # Assuming X_train is a DataFrame

# Create a DataFrame for visualization
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importance})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Plot
plt.figure(figsize=(10, 50))
plt.barh(importance_df['Feature'], importance_df['Importance'])
plt.xlabel('Importance')
plt.title('Feature Importance')
plt.show()


In [ ]:
import shap

#Create a SHAP explainer
explainer = shap.TreeExplainer(best_model)

# Calculate SHAP values
shap_values = explainer.shap_values(X)

# Summary Plot
shap.summary_plot(shap_values, X)

# Force Plot for the first instance
shap.force_plot(explainer.expected_value, shap_values[0], X.iloc[0])

# Dependence Plot for a specific feature (replace 'feature_name' with the actual feature name)
shap.dependence_plot('feature_name', shap_values, X)


